In [225]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import torch
import theseus as th
from torch.functional import F

import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

In [228]:
class Config:
    img_width: int = 640
    img_height: int = 640
    focal_length: int = (img_width * 0.5) / np.tan(60.0 * np.pi / 180.0);
conf = Config()

def get_random_image_point(conf: Config = conf):
    x = random.uniform(0, conf.img_width)
    y = random.uniform(0, conf.img_height)
    x = np.array([x, y], dtype=np.float32)    
    return x

def to_homogeneous(x):
    return np.concatenate([x, np.ones(1)])

def to_camera_coords(x: torch.tensor, conf: Config = conf):
    x = to_homogeneous(x)
    
    x[0] -= conf.img_width // 2
    x[1] -= conf.img_height // 2
    x[:2] /= conf.focal_length
    x /= x.norm()
    
    return x

In [231]:
xs = np.stack([get_random_image_point() for _ in range(100)]

In [232]:
xs

array([[376.48083 , 416.77316 ],
       [302.79883 , 220.25256 ],
       [130.82332 ,  24.886974],
       [212.74324 , 218.30495 ],
       [499.52823 , 307.70575 ],
       [356.53754 ,  32.64478 ],
       [240.8237  , 126.23172 ],
       [392.41403 , 490.63132 ],
       [287.39124 , 607.38776 ],
       [169.07048 , 403.2044  ],
       [621.678   , 470.7514  ],
       [632.20917 , 460.7857  ],
       [527.2516  , 606.6787  ],
       [407.90848 , 313.50674 ],
       [610.11786 , 546.31866 ],
       [422.31467 , 579.18726 ],
       [ 28.152441, 192.77153 ],
       [ 18.031166, 586.9388  ],
       [175.5578  , 495.43732 ],
       [608.73694 , 139.83015 ],
       [573.10956 , 505.1883  ],
       [297.61182 , 439.83997 ],
       [ 74.61515 ,  81.56889 ],
       [196.9258  , 474.69156 ],
       [553.50415 ,  31.096193],
       [446.41168 , 378.12408 ],
       [347.17462 , 108.97014 ],
       [123.38906 , 168.48222 ],
       [205.29385 , 270.54645 ],
       [368.652   , 635.6788  ],
       [61

In [233]:
xs.shape

(100, 2)

In [234]:
H = np.array([
    [1.243715, -0.461057, -111.964454], 
    [0.0,       0.617589, -192.379252],
    [0.0,      -0.000983,    1.0]
])

In [235]:
H

array([[   1.243715,   -0.461057, -111.964454],
       [   0.      ,    0.617589, -192.379252],
       [   0.      ,   -0.000983,    1.      ]])

In [236]:
xs1 = np.concatenate((xs, np.ones((xs.shape[0], 1))), axis=1)

In [237]:
xs2 = (H @ xs1.T).T
xs2.shape

(100, 3)

In [238]:
xs2 = xs2[:, :2] / xs2[:, 2][:, None]

In [239]:
xs2

array([[ 278.01269298,  110.13713251],
       [ 208.14770354,  -71.92633873],
       [  40.25289853, -181.44826177],
       [  66.17783106,  -73.28248163],
       [ 526.7714799 ,   -3.35982944],
       [ 326.90587439, -177.92786836],
       [ 147.67607721, -130.62915041],
       [ 289.50170923,  213.6898601 ],
       [ -85.79993601,  453.51101076],
       [-145.09925586,   93.82149914],
       [ 826.76856276,  183.0644689 ],
       [ 844.30505181,  168.53546647],
       [ 654.23687261,  451.64300538],
       [ 362.53927151,    1.79101445],
       [ 853.11207722,  313.24175738],
       [ 339.56405823,  383.87785694],
       [-204.60007566,  -90.46905628],
       [-851.34208671,  402.10858072],
       [-237.91221862,  221.44382211],
       [ 673.19365097, -122.91700267],
       [ 730.83029126,  237.62316834],
       [  97.57732586,  139.63330238],
       [ -61.72133528, -154.38188314],
       [-161.05786366,  188.95604202],
       [ 579.82363542, -178.63502145],
       [ 427.98977014,   

In [240]:
xs2 += np.random.normal(0, 0.1, xs2.shape[0] * 2).reshape(xs2.shape[0], 2)

In [241]:
xs2.shape
xs2

array([[ 278.05813562,  110.03722885],
       [ 208.04993282,  -72.00898418],
       [  40.379001  , -181.50587716],
       [  66.28806159,  -73.36551342],
       [ 526.69514835,   -3.24375438],
       [ 326.70018879, -177.94820098],
       [ 147.68825095, -130.67785266],
       [ 289.5194848 ,  213.74779653],
       [ -85.71149319,  453.46135144],
       [-145.01998547,   93.89259896],
       [ 826.88032889,  183.1759601 ],
       [ 844.12422182,  168.594559  ],
       [ 654.41177141,  451.52176556],
       [ 362.52518145,    1.93286712],
       [ 853.28945928,  313.23970691],
       [ 339.59006136,  383.93312949],
       [-204.43213002,  -90.51235083],
       [-851.26841806,  402.2028309 ],
       [-238.02329038,  221.50198807],
       [ 673.21394175, -122.93584913],
       [ 730.65688812,  237.62091863],
       [  97.81014328,  139.62180784],
       [ -61.72825771, -154.30183131],
       [-161.08188929,  188.80706954],
       [ 579.70609466, -178.57151315],
       [ 427.95242866,   

In [242]:
matrix, mask = cv2.findHomography(xs1, xs2, cv2.RANSAC, 5.0)

In [243]:
matrix

array([[   1.24355467,   -0.46104798, -111.91408068],
       [   0.00000876,    0.61753791, -192.36015808],
       [  -0.00000004,   -0.00098304,    1.        ]])

In [244]:
num, Rs, Ts, Ns = cv2.decomposeHomographyMat(
    H,
    np.array([
        [conf.focal_length, 0., conf.img_width // 2],
        [0., conf.focal_length, conf.img_height // 2],
        [0., 0., 1]
    ])
)

In [245]:
# homography decomposition: https://docs.opencv.org/3.4/de/d45/samples_2cpp_2tutorial_code_2features2D_2Homography_2decompose_homography_8cpp-example.html#a20
def from_decomposed(R, t, n):
    H = R + t @ n.T
    H /= H[2, 2]
    
    return H

In [246]:
import torch

# x
def get_roll_mtx(roll: float, device: torch.device, dtype: torch.dtype):
    roll = torch.tensor(roll)
    roll = torch.deg2rad(roll)
    return torch.tensor([[1., 0., 0.],
                           [0., torch.cos(roll), -torch.sin(roll)],
                           [0., torch.sin(roll), torch.cos(roll)]],
                         dtype=dtype, 
                         device=device) 

# y
def get_pitch_mtx(pitch: float, device: torch.device, dtype: torch.dtype):
    pitch = torch.tensor(pitch)
    pitch = torch.deg2rad(pitch)
    return torch.tensor([[torch.cos(pitch), 0., torch.sin(pitch)],
                           [0., 1., 0.],
                           [-torch.sin(pitch), 0., torch.cos(pitch)]],
                         dtype=dtype)

# z
def get_yaw_mtx(yaw: float, device: torch.device, dtype: torch.dtype):
    yaw = torch.tensor(yaw)
    yaw = torch.deg2rad(yaw)
    return torch.tensor([[torch.cos(yaw), -torch.sin(yaw), 0.],
                           [torch.sin(yaw), torch.cos(yaw), 0.],
                           [0., 0., 1.]],
                         dtype=dtype)

# z, y, x
def get_rt_mtx(yaw: float, pitch: float, roll: float,
               device: torch.device, dtype: torch.dtype):
    RZ = get_yaw_mtx(yaw, device, dtype)
    RY = get_pitch_mtx(pitch, device, dtype)
    RX = get_roll_mtx(roll, device, dtype)

    # roll, pitch, yaw

    R = RX @ (RY @ RZ)

    return R

In [247]:
print(H)
print()

dev_x = 0.1
dev_z = 0.2


# solution can be eliminated by applying one more
# point correspondence and depth constraint
for sol in range(num):
    R, t, n = Rs[sol], Ts[sol], Ns[sol]
        
    Rnoise = get_rt_mtx(dev_x, 0, dev_z, "cpu", torch.float64).numpy()
    Rnoised = R @ Rnoise
    tnoised = Rnoise @ t
    
    dec = from_decomposed(R, t, n)
    dec_noise = from_decomposed(Rnoised, tnoised, n)

    # print(R, t, n, "\n")
    # print(dec)
    # print(dec_noise)
    # print()

[[   1.243715   -0.461057 -111.964454]
 [   0.          0.617589 -192.379252]
 [   0.         -0.000983    1.      ]]



In [248]:
# Works nice only up to 2 degrees of deviation around all the axes
# In this case the deviation can be seen only as pixel noise
# https://ceres-solver.googlesource.com/ceres-solver/+/master/examples/libmv_homography.cc
# 
# To fix this, it would be natural to try optimization over manifold of rotation directly as done
# in Colmap:
# https://github.com/colmap/colmap/blob/bf3e19140f491c3042bfd85b7192ef7d249808ec/src/estimators/pose.cc
def symmetric_geometric_distance_terms(H, x1, x2):
    # to homogeneous
    x1 = torch.cat([x1, torch.ones(1, x1.shape[1])], axis=0)
    x2 = torch.cat([x2, torch.ones(1, x2.shape[1])], axis=0)
    
    H_x = H @ x1
    Hinv_y = torch.linalg.inv(H) @ x2
    
    H_x = H_x / H_x[2, :]
    Hinv_y = Hinv_y / Hinv_y[2, :]

    return H_x, Hinv_y

def experiment(dev_x, dev_z): 
    global H
    global xs1, xs2
    
    num, Rs, Ts, Ns  = cv2.decomposeHomographyMat(H, np.eye(3))
    R, t, n = Rs[0], Ts[0], Ns[0]
    Rnoise = get_rt_mtx(dev_x, 0, dev_z, "cpu", torch.float64).numpy()
    Rnoised = R @ Rnoise
    tnoised = Rnoise @ t
    
    dec = from_decomposed(R, t, n)
    dec_noise = from_decomposed(Rnoised, tnoised, n)
    
    Horig = torch.tensor(dec)
    Hnoised = torch.tensor(dec_noise)
    
    xs1 = torch.tensor(xs1[:, :2], dtype=torch.float64, requires_grad=False)
    xs2 = torch.tensor(xs2, dtype=torch.float64, requires_grad=False)

    weights_LBFGS = torch.tensor(Hnoised.clone().to(torch.float64), requires_grad=True)
    weights = weights_LBFGS

    optimizer = torch.optim.LBFGS([weights_LBFGS], max_iter=10000, lr=0.1)
    losses = []

    global best_loss
    global best_guess
    best_guess, best_loss = None, None

    def closure():
        optimizer.zero_grad()
        H_x, Hinv_y = symmetric_geometric_distance_terms(weights, xs1.T, xs2.T)
        loss1 = F.mse_loss(H_x[:2, :], xs2.T)
        loss2 = F.mse_loss(Hinv_y[:2, :], xs1.T)
        loss = (loss1 + loss2) / 100
        loss.backward()

        global best_loss
        global best_guess

        if best_loss is None or loss < best_loss:
            best_guess = weights.clone()
            best_loss = loss
        losses.append(loss.clone())
        
        return loss

    for i in range(10):
        optimizer.step(closure)

    best_guess = best_guess / best_guess[2, 2]    
    print("Optim: ", best_guess)
    print("Horig: ", Horig)

    plt.plot([elm.detach().numpy() for elm in losses])

In [153]:
experiment(0.5, 0.2)


KeyboardInterrupt



## Let's check that the implementation is ok and use Theseus to do bacically same thing

In [312]:
from typing import Tuple

def homography_symmetric_error_fn(
    optim_vars: Tuple[th.Manifold],
    aux_vars: Tuple[th.Variable]
) -> torch.Tensor:
    
    H = optim_vars[0].tensor.reshape((-1, 9))
    H = H.reshape((3, 3))
    xs1, xs2 = aux_vars # pixel coords
    
    H_xs1, Hinv_xs2 = symmetric_geometric_distance_terms(H, xs1.tensor[0, :].T, xs2.tensor[0, :].T)
    
    proj_err_d = (H_xs1[:2, :] - xs2.tensor[0, :].T) 
    proj_err_inv = (Hinv_xs2[:2, :] - xs1.tensor[0, :].T)

    proj_err = proj_err_d + proj_err_inv
    # proj_err = torch.sum(torch.sum(proj_err, axis=-1, keepdim=False), axis=-1, keepdim=True)[None, :]

    proj_err = torch.mean(proj_err, axis=0, keepdim=True)
    return proj_err

In [313]:
def experiment2(dev_x, dev_z): 
    global H
    global xs1, xs2
    
    num, Rs, Ts, Ns  = cv2.decomposeHomographyMat(H, np.eye(3))
    R, t, n = Rs[0], Ts[0], Ns[0]
    Rnoise = get_rt_mtx(dev_z, 0, dev_x, "cpu", torch.float64).numpy()
    Rnoised = Rnoise @ R
    tnoised = Rnoise @ t
    nnoised = Rnoise @ n
    
    from scipy.spatial.transform import Rotation
    print(Rotation.from_matrix(Rnoised).as_euler("XYZ", degrees=True))
    print(Rotation.from_matrix(R).as_euler("XYZ", degrees=True))
    
    dec = from_decomposed(R, t, n)
    dec_noise = from_decomposed(Rnoised, t, n)
    
    Horig = torch.tensor(dec)
    Hnoised = torch.tensor(dec_noise)
    
    print(Hnoised)

    xs1 = torch.tensor(xs1[:, :2], dtype=torch.float64, requires_grad=False)
    xs2 = torch.tensor(xs2, dtype=torch.float64, requires_grad=False)
    
    x = th.Variable(xs1.to(torch.float32)[None, :], name="FirstImgKpts")
    y = th.Variable(xs2.to(torch.float32)[None, :], name="SecondImgKpts")
    H0 = th.Vector(tensor=torch.tensor(Hnoised, dtype=torch.float32).ravel()[None, :], name="Hest")
    
    cost_function = th.AutoDiffCostFunction(
        [H0],
        homography_symmetric_error_fn,
        100,
        aux_vars=[x, y],
        name="homography_symmetric_error_fn"
    )
    
    objective = th.Objective()
    objective.add(cost_function)
    optimizer = th.LevenbergMarquardt(
        objective,
        max_iterations=1000,
        step_size=1,
        abs_err_tolerance=1e-12, 
        rel_err_tolerance=1e-5
    )
    theseus_optim = th.TheseusLayer(optimizer)
    
    theseus_inputs = {
        "xs1": x,
        "xs2": y,
        "H": th.Vector(tensor=torch.tensor(Hnoised, dtype=torch.float32).ravel()[None, :]),
    }
 
    with torch.no_grad():
        updated_inputs, info = theseus_optim.forward(
            theseus_inputs, 
            optimizer_kwargs={"track_best_solution": True, "verbose": False, "track_err_history": True}
        )

    if info.best_solution is not None:
        Hrec = info.best_solution["Hest"].reshape((-1, 9))
        Hrec = Hrec.reshape((3, 3))
        Hrec = Hrec / Hrec[2, 2]
        
    print(info, '\n')
        
    print(Hrec, '\n', Horig)

In [314]:
experiment2(0, 15)

[ 0.0021952  -0.00433598 18.21395654]
[ 0.00099817 -0.00475639  3.2139565 ]
tensor([[     1.1812,     -0.7914,   -111.9644],
        [     0.3303,      0.5551,   -192.3793],
        [     0.0000,     -0.0010,      1.0000]], dtype=torch.float64)
NonlinearOptimizerInfo(best_solution={'Hest': tensor([[     2.0041,     -0.7481,   -178.0673,      0.0001,      0.9978,
           -310.2282,     -0.0000,     -0.0016,      1.6134]])}, status=array([<NonlinearOptimizerStatus.CONVERGED: 1>], dtype=object), converged_iter=tensor([42]), best_iter=tensor([34]), err_history=tensor([[708733.3750, 191042.7188,   4626.4512,  ...,         inf,
                 inf,         inf]]), last_err=tensor([0.0627]), best_err=tensor([0.0627]), state_history=None) 

tensor([[     1.2422,     -0.4637,   -110.3704],
        [     0.0001,      0.6184,   -192.2870],
        [    -0.0000,     -0.0010,      1.0000]]) 
 tensor([[     1.2437,     -0.4611,   -111.9645],
        [     0.0000,      0.6176,   -192.3793],
     